# Using Naive Bayes algorithm for sms spam detection

In this assigment, you will predict if a sms message is 'spam' or 'ham' (i.e. not 'spam') using the Bernoulli Naive Bayes *classifier*.

The training data is from the UCI Machine Learning Repository: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection.  Please download the zip file before running the code below. 


## Step 1:  Getting, understanding, and cleaning the dataset


###  Importing the dataset

In [1]:
# Import the usual libraries
import numpy as np 
import pandas as pd  # To read in the dataset we will use the Panda's library
df = pd.read_table('SMSSpamCollection', sep = '\t', header=None, names=['label', 'sms_message'])

# Next we observe the first 5 rows of the data to ensure everything was read correctly
df.head()


,label,sms_message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


### Data preprocesssing
It would be more convenient if the labels were binary instead of 'ham' and 'spam'.  This way our code can always work with numerical values instead of strings.

In [2]:
df['label']=df.label.map({'spam':1, 'ham':0})
df.head() # Again, lets observe the first 5 rows to make sure everything worked before we continue

,label,sms_message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


### Splitting the dcoument into training set and test set

In [3]:
# This time we will use sklearn's method for seperating the data
from sklearn.model_selection import train_test_split

df_train_msgs, df_test_msgs, df_ytrain, df_ytest = train_test_split(df['sms_message'],df['label'], random_state=0)

#Looking at the train/test split
print("The number of training examples: ", df_train_msgs.shape[0])
print("The number of test exampels: ", df_test_msgs.shape[0])

print("The first four labels")
print(df_ytrain[0:4])

print("The first four sms messages")
print(df_train_msgs[0:4])


The number of training examples:  4179
The number of test exampels:  1393
The first four labels
872     0
831     1
1273    0
3314    0
Name: label, dtype: int64
The first four sms messages
872     Its going good...no problem..but still need li...
831     U have a secret admirer. REVEAL who thinks U R...
1273                                                Ok...
3314    Huh... Hyde park not in mel ah, opps, got conf...
Name: sms_message, dtype: object


###  Creating the feature vector from the text (feature extraction)

Each message will have its own feature vector.  For each message we will create its feature vector as we discussed in class; we will have a feature for every word in our vocabulary.  The $j$th feature is set to one ($x_j=1$) if the $j$th word from our vocabulary occurs in the message, and set the $j$ feature is set to $0$ otherwise ($x_j=0$).

We will use the sklearn method CountVectorize to create the feature vectors for every messge.

We could have written the code to do this directly by performing the following steps:
* remove capitalization
* remove punctuation 
* tokenize (i.e. split the document into individual words)
* count frequencies of each token 
* remove 'stop words' (these are words that will not help us predict since they occur in most documents, e.g. 'a', 'and', 'the', 'him', 'is' ...

In [4]:
# importing the library
from sklearn.feature_extraction.text import CountVectorizer
# creating an instance of CountVectorizer

# Note there are issues with the way CountVectorizer removes stop words.  To learn more: https://scikit-learn.org/stable/modules/feature_extraction.html#stop-words
vectorizer = CountVectorizer(binary = True, stop_words='english')

# If we wanted to perform Multnomial Naive Bayes, we would include the word counts and use the following code
#vectorizer = CountVectorizer(stop_words='english')

# To see the 'count_vector' object
print(vectorizer)

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)


In [5]:
# To see the 'stop words' 
# print(vectorizer.get_stop_words())

In [6]:
# Create the vocabulary for our feature transformation
vectorizer.fit(df_train_msgs)

# Next we create the feature vectors for both the training data and the test data
X_train = vectorizer.transform(df_train_msgs).toarray() # code to turn the training emails into a feature vector
X_test = vectorizer.transform(df_test_msgs).toarray() # code to turn the test email into a feature vector

# Changing the target vectors data type  
y_train = df_ytrain.to_numpy() # Convereting from a Panda series to a numpy array
y_test = df_ytest.to_numpy()

# To observe what the data looks like 
print("The label of the first training example: ", y_train[0])
#print("The first training example: ", X_train[0].tolist())# I needed to covernt the datatype to list so all the feature values would be printed

The label of the first training example:  0


# Your code goes here

### Estimated value for P(y) when y=0 and y=1

In [7]:
# You should NOT use the features of sklearn library in your code.
#### TO-DO #####
def countbinary(arr):
    binary, counts = np.unique(arr, return_counts=True)
    return counts[0], counts[1], len(arr)

countzero, countone, counttotal = countbinary(y_train)
py0 = countzero/counttotal
py1 = countone/counttotal
print('P(y=0) =',py0)
print('P(y=1) =',py1 )

P(y=0) = 0.8655180665230916
P(y=1) = 0.13448193347690834


### Feature Admirer when y=0 and y=1

In [8]:
def countattribute(attribute, arry, arrx):
    index = vectorizer.vocabulary_[attribute]
    attribute0, attribute1 = 0, 0
    for i in range(len(arry)):
        # given y is 0, count 
        if arry[i] == 0:
            attribute0 += arrx[i][index]
        else:
            attribute1 += arrx[i][index]
    countzero, countone, counttotal = countbinary(arry)
    phi0 = attribute0/countzero
    phi1 = attribute1/countone
    return phi0, phi1

In [23]:
phi0, phi1 = countattribute('admirer', y_train, X_train)
print('phi admirer|y=0 =',phi0)
print('phi admirer|y=1 =',phi1)

phi admirer|y=0 = 0.0
phi admirer|y=1 = 0.014234875444839857


### Feature Secret when y=0 and y=1

In [10]:
phi0, phi1 = countattribute('secret', y_train, X_train)
print('phi secret|y=0 =',phi0)
print('phi secret|y=1 =',phi1 )

phi secret|y=0 = 0.000552944429084877
phi secret|y=1 = 0.014234875444839857


### Estimate φi|y for the Bernoulli corresponding to p(xi|y), for each attribute xi and each class y.

In [11]:
ham = X_train[y_train == 0]
spam = X_train[y_train == 1]
phiham = np.sum(ham, axis=0)/countzero
phispam = np.sum(spam, axis=0)/countone
print('ham:', phiham)
print('spam:', phispam)

ham: [0.         0.         0.00027647 ... 0.00027647 0.         0.00027647]
spam: [0.01245552 0.03558719 0.         ... 0.         0.00177936 0.        ]


### Prediction 

Slower loop solution, no m smoothing

In [12]:
predicted = []
for i in range(len(X_test)):
    p_map0 = np.log(py0)
    p_map1 = np.log(py1) 
    for j in range(len(X_test[0])):
        if X_test[i][j] == 0:
            p_map0 += np.log(1-phiham[j]) 
            p_map1 += np.log(1-phispam[j])
        if X_test[i][j] == 1:
            #if we encounter log(0), ignore
            if phiham[j] > 0: 
                p_map0 +=  np.log(phiham[j]) 
            if phispam[j] > 0:
                p_map1 += np.log(phispam[j])
    predicted.append(0) if p_map0 > p_map1 else predicted.append(1)

Numpy solution, with m smoothing

In [13]:
def msmooth(num, denom):
    m = 0.001
    return (num+m)/(denom+m*2)

mphiham = msmooth(ham.sum(axis=0), countzero) 
mphispam = msmooth(spam.sum(axis=0), countone)

predicted_msmooth = []
y0 = np.zeros(len(X_test)) + np.log(py0)
y1 = np.zeros(len(X_test)) + np.log(py1)
logphiham = np.where(mphiham > 0, np.log(mphiham), 0)
logphispam = np.where(mphispam > 0, np.log(mphispam), 0)
xy0 = np.where(X_test==1, logphiham, np.log(1-mphiham))
xy1 = np.where(X_test==1, logphispam, np.log(1-mphispam))     
y0 += np.sum(xy0, axis = 1)
y1 += np.sum(xy1, axis = 1)
predicted_msmooth = np.where(y0 > y1, 0, 1)

### Predicted class, no m smoothing
First 50 Examples

In [14]:
predicted = np.array(predicted)
predicted[:50]

array([1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 1, 1, 1])

First 5 Examples

In [15]:
predicted[:5]

array([1, 0, 1, 0, 1])

Last 5 Examples

In [16]:
predicted[-5:]

array([1, 0, 0, 0, 1])

### Accuracy Calculation

In [17]:
def accuracy(arr1, arr2):
    result = np.where(arr1==arr2, 1, 0)
    return countbinary(result)

No m smoothing

In [18]:
countfalse, counttrue, counttotal = accuracy(predicted, y_test)
print("Number of test examples classiﬁed correctly: ", counttrue)
print("Number of test examples classiﬁed incorrectly: ", countfalse)
print("Error Rate: ", countfalse/counttotal)
print("Accuracy Rate: ", counttrue/counttotal)

Number of test examples classiﬁed correctly:  550
Number of test examples classiﬁed incorrectly:  843
Error Rate:  0.6051687006460876
Accuracy Rate:  0.3948312993539124


With m smoothing

In [19]:
countfalse, counttrue, counttotal = accuracy(predicted_msmooth, y_test)
print("Number of test examples classiﬁed correctly: ", counttrue)
print("Number of test examples classiﬁed incorrectly: ", countfalse)
print("Error Rate: ", countfalse/counttotal)
print("Accuracy Rate: ", counttrue/counttotal)

Number of test examples classiﬁed correctly:  1377
Number of test examples classiﬁed incorrectly:  16
Error Rate:  0.01148600143575018
Accuracy Rate:  0.9885139985642498


Sanity check with ML library

In [20]:
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB(alpha=0.001)
clf.fit(X_train, y_train)
predicted = clf.predict(X_test)
countfalse, counttrue, counttotal = accuracy(predicted, y_test)
print("Number of test examples classiﬁed correctly: ", counttrue)
print("Number of test examples classiﬁed incorrectly: ", countfalse)
print("Error Rate: ", countfalse/counttotal)
print("Accuracy Rate: ", counttrue/counttotal)

Number of test examples classiﬁed correctly:  1377
Number of test examples classiﬁed incorrectly:  16
Error Rate:  0.01148600143575018
Accuracy Rate:  0.9885139985642498


### Zero-R Evaluation

In [21]:
len(ham)>len(spam)

True

In [22]:
countfalse, counttrue, counttotal = accuracy(np.zeros(len(y_test)), y_test)
print("Number of test examples classiﬁed correctly: ", counttrue)
print("Number of test examples classiﬁed incorrectly: ", countfalse)
print("Error Rate: ", countfalse/counttotal)
print("Accuracy Rate: ", counttrue/counttotal)

Number of test examples classiﬁed correctly:  1208
Number of test examples classiﬁed incorrectly:  185
Error Rate:  0.13280689160086145
Accuracy Rate:  0.8671931083991385
